In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Parameters
SPEED_THRESHOLD = 4  # m/s
MIN_STOP_DURATION = 1.0  # seconds
DIST_THRESH = 1
EXPORT_INTERVAL = 0.16  # seconds
PED_BOX = (0.3, 0.3)
MOTO_BOX = (1.87, 0.64)

# ----------------------------------
# Helpers
# ----------------------------------

def filter_stopped_vehicles(df, speed_col='speed', time_col='TimeStamp'):
    """Filter out stopped vehicles longer than threshold"""
    df = df.copy()
    df['time_diff'] = df.groupby('Track ID')[time_col].diff()
    df['is_stopped'] = df[speed_col] < SPEED_THRESHOLD
    stop_groups = (df['is_stopped'] != df['is_stopped'].shift()).cumsum()
    df['stop_duration'] = df.groupby(['Track ID', stop_groups])['time_diff'].cumsum()
    return df[~((df['is_stopped']) & (df['stop_duration'] >= MIN_STOP_DURATION))].drop(columns=['time_diff', 'is_stopped', 'stop_duration'])

def get_rotated_corners(x, y, heading, length, width):
    """Return coordinates of rotated bounding box corners"""
    half_l, half_w = length / 2, width / 2
    corners = np.array([
        [ half_l,  half_w],
        [ half_l, -half_w],
        [-half_l, -half_w],
        [-half_l,  half_w]
    ])
    rad = np.radians(heading)
    rot = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad),  np.cos(rad)]
    ])
    return (corners @ rot.T) + np.array([x, y])

def get_closest_corners(row):
    """Get direction vector and min distance between closest corners"""
    ped_corners = get_rotated_corners(row['x_smooth_ped'], row['y_smooth_ped'], row['HA_ped'], *PED_BOX)
    moto_corners = get_rotated_corners(row['x_smooth_moto'], row['y_smooth_moto'], row['HA_moto'], *MOTO_BOX)
    dists = cdist(ped_corners, moto_corners)
    idx = np.unravel_index(np.argmin(dists), dists.shape)
    min_dist = dists[idx]
    direction_vec = moto_corners[idx[1]] - ped_corners[idx[0]]
    return min_dist, direction_vec

def calculate_attc(row):
    """Compute ATTC using projection of relative motion on direction vector"""
    min_dist, direction_vec = get_closest_corners(row)
    norm = np.linalg.norm(direction_vec)
    if norm == 0:
        return np.inf
    unit_vec = direction_vec / norm

    rel_v = np.array([
        row['vx_smooth_ped'] - row['vx_smooth_moto'],
        row['vy_smooth_ped'] - row['vy_smooth_moto']
    ])
    rel_a = 0.5 * np.array([
        row['ax_ped'] - row['ax_moto'],
        row['ay_ped'] - row['ay_moto']
    ])

    closing_rate = -np.dot(rel_v, unit_vec) + np.dot(rel_a, unit_vec)
    return min_dist / closing_rate if closing_rate > 0 else np.inf

# ----------------------------------
# Load Data
# ----------------------------------

df_ped = pd.read_csv(r"D:\T\test_codeEVT\nd\ped_smooth.csv")
df_moto = pd.read_csv(r"D:\T\test_codeEVT\nd\moto_smooth.csv")
df_moto['speed'] = np.sqrt(df_moto['vx_smooth']**2 + df_moto['vy_smooth']**2)

# Filter stopped vehicles
print(f"Original motorcycle count: {len(df_moto)}")
df_moto = filter_stopped_vehicles(df_moto, speed_col='speed')
print(f"Filtered motorcycle count: {len(df_moto)}")

print(df_moto['speed'].mean())
# Calculate yaw rates
for df in [df_ped, df_moto]:
    df['yaw_rate'] = df.groupby('Track ID').apply(
        lambda x: x['HA'].diff() / x['TimeStamp'].diff()
    ).reset_index(level=0, drop=True)

# Round timestamps
df_ped['Time_rounded'] = (df_ped['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL
df_moto['Time_rounded'] = (df_moto['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL


Original motorcycle count: 307259
Filtered motorcycle count: 77484
6.785159255181011


C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_11548\3215455798.py:88: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(
C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_11548\3215455798.py:88: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(


In [12]:

# ----------------------------------
# Merge and Process
# ----------------------------------

merged = pd.merge(
    df_ped, 
    df_moto, 
    on='Time_rounded', 
    suffixes=('_ped', '_moto')
)

merged['Center_dist'] = np.hypot(
    merged['x_smooth_ped'] - merged['x_smooth_moto'],
    merged['y_smooth_ped'] - merged['y_smooth_moto']
)

# Filter by center threshold
results = merged[merged['Center_dist'] <= DIST_THRESH].copy()

# Calculate ATTC using corrected method
results['ATTC'] = results.apply(calculate_attc, axis=1)
results[['mindis', 'direction_vec']] = results.apply(
    lambda row: pd.Series(get_closest_corners(row)),
    axis=1
)
# Output
output = results[['Track ID_ped', 'Track ID_moto', 'TimeStamp_ped', 'ATTC']].rename(columns={
    'Track ID_ped': 'Ped_ID',
    'Track ID_moto': 'Moto_ID',
    'TimeStamp_ped': 'TimeStamp'
})
print(output)
len(results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_moto']].drop_duplicates())

        Ped_ID  Moto_ID  TimeStamp      ATTC
1843       282      302     106.84  0.080488
1854       282      302     107.16  0.154317
1855       282      302     107.16  0.426804
5202       381      407     130.16       inf
5203       381      407     130.32       inf
...        ...      ...        ...       ...
368384   15693    15756    5396.60       inf
368389   15693    15756    5396.76       inf
368396   15693    15756    5396.92       inf
368401   15693    15756    5397.08  1.932371
368407   15693    15756    5397.24       inf

[653 rows x 4 columns]


106

In [13]:
len(results[results['ATTC']<1][['Track ID_ped', 'Track ID_moto']].drop_duplicates())

106

In [14]:
# Create new DataFrame with selected and renamed columns
conflict_summary = results[[
    'Track ID_ped',        # Pedestrian ID
    'Track ID_moto',       # Vehicle ID
    'Type_moto',           # Vehicle type
    'TimeStamp_ped',       # Timestamp
    'mindis',              # Minimum corner distance
    'ATTC'                 # ATTC value
]].copy()

# Rename columns
conflict_summary.rename(columns={
    'Track ID_ped': 'PED_ID',
    'Track ID_moto': 'VEH_ID',
    'Type_moto': 'TYPE',
    'TimeStamp_ped': 'TIMESTAMP',
    'mindis': 'MIN_COR_DIS',
    'ATTC': 'ATTC'
}, inplace=True)

# Preview
print(conflict_summary.head())


      PED_ID  VEH_ID         TYPE  TIMESTAMP  MIN_COR_DIS      ATTC
1843     282     302   Motorcycle     106.84     0.264678  0.080488
1854     282     302   Motorcycle     107.16     0.504374  0.154317
1855     282     302   Motorcycle     107.16     0.683216  0.426804
5202     381     407   Motorcycle     130.16     0.680352       inf
5203     381     407   Motorcycle     130.32     0.780840       inf


In [15]:
conflict_summary.to_csv(r"D:\T\test_codeEVT\ATTC_Data/moto_Ped.csv", index=False)